# Julia 機器學習：LIBSVM 支撐向量機

## 作業 029：

請使用 SVM 來建立一個預測模型，來對手寫數字辨識資料集，建立一個分類模型，預測數字為何。

本作業需要使用到的套件有 MLDatasets，請在執行以下範例前先安裝。

```
] add MLDatasets
```

In [18]:
import Pkg
Pkg.add("MLDatasets")
Pkg.add("HDF5")
Pkg.add("MAT")

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


In [21]:
Pkg.build("HDF5")
using HDF5, MAT
using LIBSVM, MLDatasets, Statistics

  Building CMake → `~/.julia/packages/CMake/ULbyn/deps/build.log`
  Building Blosc → `~/.julia/packages/Blosc/lzFr0/deps/build.log`
  Building HDF5 ─→ `~/.julia/packages/HDF5/Zh9on/deps/build.log`


┌ Info: Precompiling HDF5 [f67ccb44-e63f-5c2f-98bd-6dc0ccc4ba2f]
└ @ Base loading.jl:1242
┌ Info: Precompiling MAT [23992714-dd62-5051-b70f-ba57cb901cac]
└ @ Base loading.jl:1242
┌ Info: Precompiling MLDatasets [eb30cadb-4394-5ae3-aed4-317e484a6458]
└ @ Base loading.jl:1242


## 讀取資料

呼叫 MNIST 資料集的過程中，會先去檢查以前是否有下載過，如果是第一次下載，過程中會詢問是否下載資料集，請回答 `y`。

In [22]:
train_X, train_y = MNIST.traindata(1:10000)
test_X,  test_y  = MNIST.testdata(1:1000)

This program has requested access to the data dependency MNIST.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: THE MNIST DATABASE of handwritten digits
Authors: Yann LeCun, Corinna Cortes, Christopher J.C. Burges
Website: http://yann.lecun.com/exdb/mnist/

[LeCun et al., 1998a]
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner.
    "Gradient-based learning applied to document recognition."
    Proceedings of the IEEE, 86(11):2278-2324, November 1998

The files are available for download at the offical
website linked above. Note that using the data
responsibly and respecting copyright remains your
responsibility. The authors of MNIST aren't really
explicit about any terms of use, so please read the
website to make sure you want to download the
dataset.



Do you want to download the dataset from ["http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyt

(FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

...

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N

In [23]:
typeof(train_X)

Base.ReinterpretArray{FixedPointNumbers.Normed{UInt8,8},3,UInt8,Array{UInt8,3}}

In [24]:
typeof(train_y)

Array{Int64,1}

In [25]:
size(train_X)

(28, 28, 10000)

In [26]:
size(train_y)

(10000,)

In [27]:
reshape(reshape(train_X, 28*28, 10000)[:, 1], 28, 28) == train_X[:, :, 1]

true

## 資料轉換

In [28]:
train_X = reshape(train_X, 28*28, 10000)'  # convert to the shape that fits model
test_X = reshape(test_X, 28*28, 1000)'  # convert to the shape that fits model

1000×784 LinearAlgebra.Adjoint{FixedPointNumbers.Normed{UInt8,8},Base.ReshapedArray{FixedPointNumbers.Normed{UInt8,8},2,Base.ReinterpretArray{FixedPointNumbers.Normed{UInt8,8},3,UInt8,Array{UInt8,3}},Tuple{}}}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0

In [29]:
train_y = Vector(string.(train_y))  # convert to string type
test_y = Vector(string.(test_y))  # convert to string type

1000-element Array{String,1}:
 "7"
 "2"
 "1"
 "0"
 "4"
 "1"
 "4"
 "9"
 "5"
 "9"
 "0"
 "6"
 "9"
 ⋮  
 "1"
 "6"
 "2"
 "3"
 "9"
 "0"
 "1"
 "2"
 "2"
 "0"
 "8"
 "9"

In [30]:
model = LIBSVM.fit!(SVC(), train_X, train_y)

SVC(LIBSVM.Kernel.RadialBasis, 0.0012755102040816326, nothing, 1.0, 3, 0.0, 0.001, true, false, false, LIBSVM.SVM{String}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 784, 10, ["5", "0", "4", "1", "9", "2", "3", "6", "7", "8"], Int32[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], Float64[], Int32[], LIBSVM.SupportVectors{String,FixedPointNumbers.Normed{UInt8,8}}(5196, Int32[649, 327, 536, 368, 668, 522, 595, 416, 517, 598], ["5", "5", "5", "5", "5", "5", "5", "5", "5", "5"  …  "8", "8", "8", "8", "8", "8", "8", "8", "8", "8"], FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8], Int32[1, 12, 36, 66, 101, 133, 139, 174, 176, 183  …  9926, 9937, 9938, 9945, 9949, 9953, 9957, 9971, 9983, 9997], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SV

## 預測

In [31]:
ŷ = String[]
for i = 1:size(test_X, 1)
    y_ = LIBSVM.predict(model, Matrix(test_X[i, :]'))
    append!(ŷ, y_)
end

## 評估

In [32]:
using Printf
@printf("Accuracy: %.2f%%\n", mean((ŷ .== test_y))*100)

Accuracy: 89.90%
